In [96]:
import numpy as np
import tensorflow as tf

np.random.seed(3)
tf.set_random_seed(3)

In [43]:
class SumTree:
    data_pointer = 0
    def __init__(self, capacity):
        self.capacity = capacity  # for all priority values
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)  # for all transitions
    
    def add(self, p, data):
        tree_idx = self.data_pointer + self.capacity - 1
        self.data[self.data_pointer] = data  # update data_frame
        self.update(tree_idx, p)  # update tree_frame

        self.data_pointer += 1
        if self.data_pointer >= self.capacity:  # replace when exceed the capacity
            self.data_pointer = 0

    def update(self, tree_idx, p):
        change = p - self.tree[tree_idx]
        self.tree[tree_idx] = p
        # then propagate the change through tree
        while tree_idx != 0:    # this method is faster than the recursive loop in the reference code
            tree_idx = (tree_idx - 1) // 2
            self.tree[tree_idx] += change

    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for transitions
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        parent_idx = 0
        while True:     # the while loop is faster than the method in the reference code
            cl_idx = 2 * parent_idx + 1         # this leaf's left and right kids
            cr_idx = cl_idx + 1
            if cl_idx >= len(self.tree):        # reach bottom, end search
                leaf_idx = parent_idx
                break
            else:       # downward search, always search for a higher priority node
                if v <= self.tree[cl_idx]:
                    parent_idx = cl_idx
                else:
                    v -= self.tree[cl_idx]
                    parent_idx = cr_idx

        data_idx = leaf_idx - self.capacity + 1
        return leaf_idx, self.tree[leaf_idx], self.data[data_idx]

    @property
    def total_p(self):
        return self.tree[0]  # the root

In [35]:
class Memory(object):  # stored as ( s, a, r, s_ ) in SumTree
    """
    This SumTree code is modified version and the original code is from:
    https://github.com/jaara/AI-blog/blob/master/Seaquest-DDQN-PER.py
    """
    epsilon = 0.01  # small amount to avoid zero priority
    alpha = 0.6  # [0~1] convert the importance of TD error to priority
    beta = 0.4  # importance-sampling, from initial value increasing to 1
    beta_increment_per_sampling = 0.001
    abs_err_upper = 1.  # clipped abs error

    def __init__(self, capacity):
        self.tree = SumTree(capacity)

    def store(self, transition):
        max_p = np.max(self.tree.tree[-self.tree.capacity:])
        if max_p == 0:
            max_p = self.abs_err_upper
        self.tree.add(max_p, transition)   # set the max p for new p

    def sample(self, n):
        b_idx, b_memory, ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, self.tree.data[0].size)), np.empty((n, 1))
        pri_seg = self.tree.total_p / n       # priority segment
        self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])  # max = 1

        min_prob = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_p     # for later calculate ISweight
        for i in range(n):
            a, b = pri_seg * i, pri_seg * (i + 1)
            v = np.random.uniform(a, b)
            idx, p, data = self.tree.get_leaf(v)
            prob = p / self.tree.total_p
            ISWeights[i, 0] = np.power(prob/min_prob, -self.beta)
            b_idx[i], b_memory[i, :] = idx, data
        return b_idx, b_memory, ISWeights

    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.epsilon  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.abs_err_upper)
        ps = np.power(clipped_errors, self.alpha)
        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

In [171]:

class DQN:
    
    def __init__(
        self,
        num_actions,
        num_features,
        hidden_units = 128,
        learning_rate=0.0001,
        reward_discount = 0.95,
        epsilon_greedy = 0.9,
        lamda = 5,
        e_increment = None,
        replace_target_iter = 200,
        mem_size = 2000,
        batch_size = 128,
        Rmax = 15,
        dueling = False,
        prioritized = True,
        output_graph = False,
        output_path = None,
        sess = None
    ):
        # network layer sizes
        self.num_actions, self.num_features = num_actions, num_features
        self.hidden_units = hidden_units
        # training 
        self.lr = learning_rate
        self.gamma = reward_discount
        self.replace_target_iter = replace_target_iter
        self.memory_size = mem_size
        self.batch_size = batch_size
        self.lamda = lamda
        # epsilon
        self.e_increment = e_increment
        self.e_max = epsilon_greedy
        self.epsilon = 0 if e_increment else (1 - self.e_max)

        self.phase = tf.placeholder(tf.bool)
        
        self.dueling = dueling
        self.prioritized = prioritized
        self.learn_step_counter = 0
        # reward
        self.rmax = Rmax
        
        # s, a, r ,s_
        if self.prioritized:
            self.memory = Memory(capacity=self.memory_size)
        else:
            self.memory = np.zeros((self.memory_size, 2 * self.num_features + self.num_actions + 1))
        
        #build net
        self.__build_net__()
        # sync target network
        self.sync_target_op = [tf.assign(t, e) for t, e in zip(tf.get_collection('target_net_params'), \
                                                               tf.get_collection('eval_net_params'))]
        
        if not sess:
            self.sess = tf.Session()
        else:
            self.sess = sess
        
        if output_graph:
            if not output_path: output_path = ''
            else: output_path += '/'
            self.writer = tf.summary.FileWriter("logs/" + output_path, self.sess.graph)
        
        self.sess.run(tf.global_variables_initializer())
        
    def __build_net__(self):
        
        def build_layers(s, c_names, summary=True):
            # use xavier_initializer with normal distribution
            w_init, b_init = tf.contrib.layers.xavier_initializer(uniform=False), tf.constant_initializer(.1)
            # inpyt layer + relu
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1', [self.num_features, self.hidden_units], initializer=w_init, collections=c_names)
                b1 = tf.get_variable('b1', [1, self.hidden_units], initializer=b_init, collections=c_names)
                l1 = tf.nn.relu(tf.matmul(s, w1) + b1)
            
#             with tf.variable_scope('l1_bn_ac'):
#                 l1_bn = tf.layers.batch_normalization(l1, training=self.phase)
#                 l1_ac = tf.maximum(l1_bn, l1_bn * 0.5)
            
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [self.hidden_units, self.hidden_units], initializer=w_init, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.hidden_units], initializer=b_init, collections=c_names)
                l2 = tf.nn.relu(tf.matmul(l1, w2) + b2)
            
#             with tf.variable_scope('l2_bn_ac'):
#                 l2_bn = tf.layers.batch_normalization(l2, training=self.phase)
#                 l2_ac = tf.maximum(l2_bn, l2_bn * 0.5)
            
            if self.dueling:
                # state value
                with tf.variable_scope('Value'):
                    w2 = tf.get_variable('w3', [self.hidden_units, 1], initializer=w_init, collections=c_names)
                    b2 = tf.get_variable('b3', [1, 1], initializer=b_init, collections=c_names)
                    self.V = tf.matmul(l2, w2) + b2
                # action value
                with tf.variable_scope('Advantage'):
                    w2 = tf.get_variable('w3', [self.hidden_units, self.num_actions], initializer=w_init, collections=c_names)
                    b2 = tf.get_variable('b3', [1, self.num_actions], initializer=b_init, collections=c_names)
                    self.A = tf.matmul(l2, w2) + b2
                # output Q value layer
                with tf.variable_scope('Q'):
                    # Q = V(s) + A(s,a)
                    out = self.V + (self.A - tf.reduce_mean(self.A, axis=1, keep_dims=True))
            else:
                # output Q value layer
                with tf.variable_scope('Q'):
                    w2 = tf.get_variable('w3', [self.hidden_unitshidden_size, self.num_actions], initializer=w_init, collections=c_names)
                    b2 = tf.get_variable('b3', [1, self.num_actions], initializer=b_init, collections=c_names)
                    out = tf.matmul(l2, w2) + b2
            
            if summary:
                tf.summary.histogram('V', self.V)
                tf.summary.histogram('A', self.A)
                tf.summary.histogram('Q', out)
            
            return out
        
        ## ------------------ build evaluate_net ------------------
        # input, i.e state
        self.s = tf.placeholder(tf.float32, [None, self.num_features], name='s')
        self.q_target = tf.placeholder(tf.float32, [None, self.num_actions], name='q_target')
        
        if self.prioritized:
            self.ISWeights = tf.placeholder(tf.float32, [None, 1], name='IS_weights')
        
        with tf.variable_scope('eval_net'):
            self.q_eval = build_layers(self.s, ['eval_net_params', tf.GraphKeys.GLOBAL_VARIABLES])
        
        with tf.variable_scope('loss'):
            # clip to +/- Rmax
            if self.prioritized:
                self.abs_errors = tf.reduce_sum(tf.abs(self.q_target - self.q_eval), axis=1)
                self.loss = tf.reduce_mean(self.ISWeights * tf.squared_difference(self.q_target, self.q_eval)) + \
                            self.lamda * tf.reduce_sum(tf.maximum(tf.abs(self.q_eval) - self.rmax, 0))

            else:
                self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval)) + \
                            self.lamda * tf.reduce_sum(tf.maximum(tf.abs(self.q_eval) - self.rmax, 0))
                    
        with tf.variable_scope('train'):
            self._train_op = tf.train.AdamOptimizer(self.lr).minimize(self.loss)
        
        with tf.variable_scope('predict'):
            self.predict = tf.argmax(self.q_eval, 1, name='predict')
        
        ## ------------------ build target_net ------------------
        self.s_ = tf.placeholder(tf.float32, [None, self.num_features], name='s_')
        
        with tf.variable_scope('target_net'):
            self.q_next = build_layers(self.s_, ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES], summary=False)
        
        ## ------------------ summary ------------------
        tf.summary.scalar('loss', self.loss)
        self.merged = tf.summary.merge_all()
        
    def store_transition(self, histories):
        if self.prioritized:
            for history in histories:
                self.memory.store(history)
        else:
            self.memory = histories
            #####################
            # if with simulator #
            #####################
            #     def store_transition(self, s, a, r, s_):
            #         if not hasattr(self, 'memory_counter'):
            #             self.memory_counter = 0
            #         transition = np.hstack((s, a, r, s_))
            #         # start to replace when full
            #         index = self.memory_counter % self.memory_size
            #         self.memory[index, :] = transition
            #         self.memory_counter += 1

            #     def choose_action(self, observation):
            #         observation = observation[np.newaxis, :]
            #         if np.random.uniform() > self.epsilon:
            #             actions_value = self.sess.run(self.q_eval, feed_dict={self.s: observation})
            #             action = np.argmax(actions_value)
            #         else:
            #             action = np.random.randint(0, self.num_actions)
            #         return action
            
    def learn(self):
        
        if self.learn_step_counter % self.replace_target_iter == 0 and self.learn_step_counter != 0:
            self.sess.run(self.sync_target_op)
            print('\ntarget_params_synced\n')
        
        if self.prioritized:
            tree_idx, batch_memory, ISWeights = self.memory.sample(self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
            batch_memory = self.memory[sample_index, :]
        
        # next observation
        q_next = self.sess.run(self.q_next, feed_dict={self.s_: batch_memory[:, -self.num_features:]})
        q_eval = self.sess.run(self.q_eval, {self.s: batch_memory[:, :self.num_features]})

        q_target = q_eval.copy()
        q_target[q_target > self.rmax] = self.rmax
        q_target[q_target < -self.rmax] = - self.rmax
        
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        eval_act_index = batch_memory[:, self.num_features].astype(int)
        reward = batch_memory[:, self.num_features + 1]
        
        q_target[batch_index, eval_act_index] = reward + self.gamma * np.max(q_next, axis=1)
        
        if self.prioritized:
            _, abs_errors, cost, summary = self.sess.run([self._train_op, self.abs_errors, self.loss, self.merged],
                                         feed_dict={self.s: batch_memory[:, :self.num_features],
                                                    self.q_target: q_target,
                                                    self.ISWeights: ISWeights })
            self.memory.batch_update(tree_idx, abs_errors) 
        else:
            _, cost, summary = self.sess.run([self._train_op, self.loss, self.merged],
                                         feed_dict={self.s: batch_memory[:, :self.num_features],
                                                    self.q_target: q_target })
        
        self.writer.add_summary(summary, self.learn_step_counter)
        
        
        if self.e_increment:
            if self.epsilon < self.e_max:
                self.epsilon += self.e_increment 
            else:
                self.epsilon = self.e_max
            
        self.learn_step_counter += 1
        
        return cost
    
    def evaluate(self, val_set):
        
        s, a, r, s_ = val_set[sample_index, :], val_set[:, self.num_features].astype(int), \
                      val_set[:, self.num_features + 1], val_set[:, -self.num_features:]
        
        q_next = self.sess.run(self.q_next, feed_dict={self.s_: s_})
        q_eval = self.sess.run(self.q_eval, {self.s: s})
        
        a_q_eval = sess.run(self.predict, feed_dict={self.s: s_})
        
        q_next = sess.run(self.q_next, feed_dict={self.s_: s_})
        
        #double_q_value = q_next[range(self.batch_size), actions_q_eval]
        q_target = r + self.gamma * np.max(q_next, axis=1)
        
        _, abs_errors, cost, summary = self.sess.run([self.abs_errors, self.loss, self.merged],
                                         feed_dict={self.s: batch_memory[:, :self.num_features],
                                                    self.q_target: q_target,
                                                    self.ISWeights: ISWeights })

In [139]:
def train(model, histories, val_histories, num_epoches=10):
    # shuffle histories
    np.random.shuffle(histories)
    # insert memory
    model.store_transition(histories[:model.memory_size,:])
    # learn
    for epoch in range(num_epoches):
        loss = model.learn()
        #if epoch % 10 == 0:
        print ( 'epoch:{}, loss:{}'.format(epoch, loss) )
        
        if epoch % 100:
            # eval
            
            

In [155]:
import numpy as np
import discretize_sepsis_actions as discretizer
import pickle as pkl

class PatientRecordProcessor:
    
    def __init__(self, raw_path, cluster_path):
        
        print ( 'loading dataset ...' )
        self.data = np.genfromtxt(raw_path, dtype=float, delimiter=',', skip_header=1)
        print ( 'loading clustered states ...' )
        self.clusters = pkl.load(open(cluster_path, 'rb'), encoding='latin1')
        print ( 'discretizing actions ...' )
        self.discretize_actions()
        self.patient_map = None
        print ( 'initialization succeed' )
        
    def discretize_actions(self):
        
        self.action_sequence, self.vaso_bins, self.iv_bins = \
        discretizer.discretize_actions(self.data[:,50], self.data[:,47])
    
    def build_patient_map(self):
        
        self.patient_map = {}
        interventions = np.setdiff1d(np.arange(47, 57), [51,52,54,55,56])
        turcated_data = np.delete(self.data, interventions, axis=1)
        turcated_data = np.delete(turcated_data, [0,1,2], axis=1)
        turcated_data = (turcated_data - np.mean(turcated_data, axis=0)) / np.std(turcated_data, axis=0)
        for i, row in enumerate(self.data):
            icuid = str(row[1])
            state_action_outcome = [self.clusters[i], self.action_sequence[i], row[50], row[47], row[-2]]
            if icuid not in self.patient_map:
                # state_id, action, outcome
                self.patient_map[icuid] = {
                    'age':row[4], 'gender':row[3], 'sa':[state_action_outcome],
                    'obser':[turcated_data[i,:]]}
            else:
                self.patient_map[icuid]['sa'].append(state_action_outcome)
                self.patient_map[icuid]['obser'].append(turcated_data[i,:])
        
        return self.patient_map
    
    def build_training_history(self):
        memory = []
        if not self.patient_map:
            print ( 'building patient map ...' )
            self.patient_map = self.build_patient_map()
        
        for _, patient in self.patient_map.items():
            
            if len(patient['sa']) <= 5:
                continue

            for i, patient_icu_stay in enumerate(patient['sa']):
                _, action, _, _, outcome = patient_icu_stay
                s_obser = patient['obser'][i]
                next_s_obser = patient['obser'][i + 1]
                
                reward = 0
                if (i + 1) == len(patient['sa']) - 1:
                    # last stay, check the outcome
                    if patient['sa'][i + 1][-1] == 0: 
                        # survived
                        reward = 15
                    else:
                        reward = -15
                
                memory.append(np.hstack((s_obser, action, reward, next_s_obser)))
                
                if reward != 0:
                    break
        return np.array(memory)

In [125]:
prefix = '../../Dataset/'
prp = PatientRecordProcessor(prefix + 'Sepsis_imp.csv', prefix + 'states_list.pkl')

loading dataset ...
loading clustered states ...
discretizing actions ...
initialization succeed


In [126]:
histories = prp.build_training_history()
np.random.shuffle(histories)

building patient map ...


In [128]:
# train, dev split
train_histories = histories[:int(0.8 * len(histories))]
test_histories = histories[int(0.8 * len(histories)):]

In [129]:
train_histories.shape, test_histories.shape

((184936, 104), (46235, 104))

In [172]:
### main
tf.reset_default_graph()

MEMORY_SIZE = train_histories.shape[0]
ACTION_SPACE = 25
FEATURES = 51
NUM_EPOCHES = 1000

## speicify output_path while tunning parameters
with tf.variable_scope('dueling'):
    dueling_DQN = DQN(
        num_actions=ACTION_SPACE, num_features=FEATURES, batch_size=32, mem_size=MEMORY_SIZE,
        dueling=True, output_graph=True)

train(dueling_DQN, train_histories, num_epoches=NUM_EPOCHES)

epoch:0, loss:0.6550407409667969
epoch:1, loss:0.2583269476890564
epoch:2, loss:0.16637790203094482
epoch:3, loss:0.1368243247270584
epoch:4, loss:0.024408211931586266
epoch:5, loss:0.24250836670398712
epoch:6, loss:0.23718848824501038
epoch:7, loss:0.43445315957069397
epoch:8, loss:0.8171199560165405
epoch:9, loss:0.37803053855895996
epoch:10, loss:0.1290515959262848
epoch:11, loss:0.12779027223587036
epoch:12, loss:0.15062865614891052
epoch:13, loss:0.22090034186840057
epoch:14, loss:0.35148221254348755
epoch:15, loss:0.5780783295631409
epoch:16, loss:0.24954169988632202
epoch:17, loss:0.10402794182300568
epoch:18, loss:0.1295078843832016
epoch:19, loss:0.37378600239753723
epoch:20, loss:0.20504017174243927
epoch:21, loss:0.21731996536254883
epoch:22, loss:0.3668731451034546
epoch:23, loss:0.551253616809845
epoch:24, loss:0.13052313029766083
epoch:25, loss:0.3195194900035858
epoch:26, loss:0.40807002782821655
epoch:27, loss:0.11454372107982635
epoch:28, loss:0.4916286766529083
epoch:

epoch:255, loss:0.13311931490898132
epoch:256, loss:0.2715395390987396
epoch:257, loss:0.0024360574316233397
epoch:258, loss:0.12720920145511627
epoch:259, loss:0.17699138820171356
epoch:260, loss:0.04841721057891846
epoch:261, loss:0.14510922133922577
epoch:262, loss:0.17057348787784576
epoch:263, loss:0.003297932678833604
epoch:264, loss:0.09684440493583679
epoch:265, loss:0.09503313153982162
epoch:266, loss:0.08394966274499893
epoch:267, loss:0.08137529343366623
epoch:268, loss:0.12038502842187881
epoch:269, loss:0.17488685250282288
epoch:270, loss:0.13977503776550293
epoch:271, loss:0.13534261286258698
epoch:272, loss:0.09795665740966797
epoch:273, loss:0.0749162957072258
epoch:274, loss:0.09135134518146515
epoch:275, loss:0.0911138728260994
epoch:276, loss:0.08520255982875824
epoch:277, loss:0.2563992738723755
epoch:278, loss:0.04892833158373833
epoch:279, loss:0.1679902821779251
epoch:280, loss:0.043504659086465836
epoch:281, loss:0.044680096209049225
epoch:282, loss:0.1231443658

epoch:487, loss:0.09851408749818802
epoch:488, loss:0.12406750023365021
epoch:489, loss:0.07502704858779907
epoch:490, loss:0.04591481387615204
epoch:491, loss:0.07099704444408417
epoch:492, loss:0.09752178192138672
epoch:493, loss:0.12365209311246872
epoch:494, loss:0.11561178416013718
epoch:495, loss:0.024396639317274094
epoch:496, loss:0.04839065670967102
epoch:497, loss:0.02438979409635067
epoch:498, loss:0.03784525766968727
epoch:499, loss:0.04688504710793495
epoch:500, loss:0.07754115760326385
epoch:501, loss:0.025441601872444153
epoch:502, loss:0.07104778289794922
epoch:503, loss:0.06701762229204178
epoch:504, loss:0.09894317388534546
epoch:505, loss:0.06800040602684021
epoch:506, loss:0.02569032832980156
epoch:507, loss:0.06782322376966476
epoch:508, loss:0.09878097474575043
epoch:509, loss:0.0490269660949707
epoch:510, loss:0.09125079959630966
epoch:511, loss:0.002742262789979577
epoch:512, loss:16.53569793701172
epoch:513, loss:0.07754848152399063
epoch:514, loss:0.0264402441

epoch:729, loss:0.05588691681623459
epoch:730, loss:0.04996220022439957
epoch:731, loss:0.07256941497325897
epoch:732, loss:0.08646894246339798
epoch:733, loss:0.055735498666763306
epoch:734, loss:0.1002625972032547
epoch:735, loss:0.08102128654718399
epoch:736, loss:0.035265423357486725
epoch:737, loss:0.05764444172382355
epoch:738, loss:0.03664056956768036
epoch:739, loss:0.019376955926418304
epoch:740, loss:0.07063455879688263
epoch:741, loss:0.05322069302201271
epoch:742, loss:0.05331617221236229
epoch:743, loss:0.0013737011468037963
epoch:744, loss:0.057674989104270935
epoch:745, loss:0.1044001579284668
epoch:746, loss:0.09250712394714355
epoch:747, loss:0.05155275762081146
epoch:748, loss:0.10321429371833801
epoch:749, loss:0.036912523210048676
epoch:750, loss:0.03713252767920494
epoch:751, loss:0.0017841275548562407
epoch:752, loss:0.03543808311223984
epoch:753, loss:0.0714135393500328
epoch:754, loss:0.0017091786721721292
epoch:755, loss:0.06781711429357529
epoch:756, loss:0.05

epoch:973, loss:0.05626550689339638
epoch:974, loss:0.05539660528302193
epoch:975, loss:0.05231013149023056
epoch:976, loss:0.03528013452887535
epoch:977, loss:0.07224494963884354
epoch:978, loss:0.020199723541736603
epoch:979, loss:0.03666631877422333
epoch:980, loss:0.05047769844532013
epoch:981, loss:0.03570803254842758
epoch:982, loss:0.05648984760046005
epoch:983, loss:0.2626574635505676
epoch:984, loss:0.05288871377706528
epoch:985, loss:0.03711981698870659
epoch:986, loss:0.07394415885210037
epoch:987, loss:0.09499017894268036
epoch:988, loss:0.0024957398418337107
epoch:989, loss:0.035993412137031555
epoch:990, loss:0.0191359780728817
epoch:991, loss:0.03681506961584091
epoch:992, loss:0.05475335195660591
epoch:993, loss:0.05860821157693863
epoch:994, loss:0.05765843391418457
epoch:995, loss:0.07555708289146423
epoch:996, loss:0.05259719491004944
epoch:997, loss:0.0679333359003067
epoch:998, loss:0.0027926659677177668
epoch:999, loss:0.0781489685177803
